In [1]:
import os
import sys
from numpy import *
sys.path.append(os.getcwd())

In [2]:
import bayes

In [3]:
# 返回实验样本和类别标签(侮辱类和非侮辱类)
listOPosts,listClasses=bayes.loadDataSet()

In [4]:
listClasses

[0, 1, 0, 1, 0, 1]

In [5]:
listOPosts

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [6]:
# 创建词汇表
# 将实验样本里面的词汇进行去重
def createVocabList(dataSet):
    vocabSet=set([])
    for document in dataSet:
        vocabSet=vocabSet|set(document)
    return list(vocabSet)

In [7]:
myVocabList=createVocabList(listOPosts)

In [8]:
myVocabList

['cute',
 'love',
 'help',
 'garbage',
 'quit',
 'I',
 'problems',
 'is',
 'park',
 'stop',
 'flea',
 'dalmation',
 'licks',
 'food',
 'not',
 'him',
 'buying',
 'posting',
 'has',
 'worthless',
 'ate',
 'to',
 'maybe',
 'please',
 'dog',
 'how',
 'stupid',
 'so',
 'take',
 'mr',
 'steak',
 'my']

In [9]:
# 将词汇转成特征向量
# 也就是将每一行样本转成特征向量
# 向量的每一元素为1或者0,分别表示词汇表中的单词在输入文档中是否出现
def setOfWordsVec(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]=1
        else:print "the word:%s is not in my Vocabulary!" % word
    return returnVec

In [10]:
setOfWordsVec(myVocabList,listOPosts[0])

[0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

## 训练算法:从词向量计算概率
朴素贝叶斯分类器训练函数

In [26]:
# 输入为文档矩阵以及由每篇文档类别标签所构成的向量
def trainNB0(trainMatrix,trainCategory):
    # 文档总数,有几篇文档,在这里也就是有几个一维数组
    numTrainDocs=len(trainMatrix)
    # 每篇文档里面的单词数,也就是一维数组的长度
    numWords=len(trainMatrix[0])
    # 因为就只有0和1两个分类,将类别列表求和后,就是其中一个类别的个数
    # 然后numTrainDocs也就是文档总数,这样相除后就是这个类别的概率了
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    # 以下两行,初始化概率
    p0Num=ones(numWords);p1Num=ones(numWords)
    p0Denom=2.0;p1Denom=2.0
    
    # 依次遍历所有的文档
    for i in range(numTrainDocs):
        # 判断这个文档所属类别
        if trainCategory[i]==1:
            # 数组与数组相加,这里就是统计每个词在这个分类里面出现的次数
            p1Num+=trainMatrix[i]
            # 统计该类别下,这些词语一共出现了多少次
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    # 通过求对数避免数据下溢出
    p1Vect=log(p1Num/p1Denom)
    p0Vect=log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

In [12]:
# 所有文档的特征向量
trainMat=[]

In [29]:
# 将文档的每一行,转成词向量,然后追加到trainMat中
for postinDoc in listOPosts:
    trainMat.append(bayes.setOfWords2Vec(myVocabList,postinDoc))

In [30]:
print ones(32)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [32]:
print len(trainMat[1])

32


In [14]:
p0V,p1V,pAb=trainNB0(trainMat,listClasses)

In [15]:
pAb

0.5

In [16]:
p0V

array([-2.56494936, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
       -2.56494936, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
       -2.15948425, -3.25809654, -3.25809654, -2.56494936, -3.25809654,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -1.87180218])

In [17]:
p1V

array([-3.04452244, -3.04452244, -3.04452244, -2.35137526, -2.35137526,
       -3.04452244, -3.04452244, -3.04452244, -2.35137526, -2.35137526,
       -3.04452244, -3.04452244, -3.04452244, -2.35137526, -2.35137526,
       -2.35137526, -2.35137526, -2.35137526, -3.04452244, -1.94591015,
       -3.04452244, -2.35137526, -2.35137526, -3.04452244, -1.94591015,
       -3.04452244, -1.65822808, -3.04452244, -2.35137526, -3.04452244,
       -3.04452244, -3.04452244])

In [18]:
# 朴素贝叶斯分类函数
def classifyNB(vec2classify,p0Vec,p1Vec,pClass1):
    #元素相乘
    p1=sum(vec2classify*p1Vec)+log(pClass1)
    p0=sum(vec2classify*p0Vec)+log(pClass1)
    if p1>p0:
        return 1
    else:
        return 0

In [24]:
def testingNB():
    listOPosts,listClasses=bayes.loadDataSet()
    myVocabList=createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWordsVec(myVocabList,postinDoc))
    
    p0V,p1V,pAb=trainNB0(array(trainMat),array(listClasses))
    
    testEntry=['love','my','dalmation']
    thisDoc=array(setOfWordsVec(myVocabList,testEntry))
    print testEntry,'classified as : ',classifyNB(thisDoc,p0V,p1V,pAb)
    
    testEntry=['stupid','garbage']
    thisDoc=array(setOfWordsVec(myVocabList,testEntry))
    print testEntry,'classified as : ',classifyNB(thisDoc,p0V,p1V,pAb)

In [25]:
testingNB()

['love', 'my', 'dalmation'] classified as :  0
['stupid', 'garbage'] classified as :  1
